In [ ]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent.parent.absolute()))

In [ ]:
import config as env_config
from multiversx_sdk import Address, CodeMetadata, TokenPayment, ProxyNetworkProvider
from multiversx_sdk.core.transaction_builders import ContractCallBuilder, ContractDeploymentBuilder, MultiESDTNFTTransferBuilder, EGLDTransferBuilder
from multiversx_sdk.core.transaction_builders import DefaultTransactionBuildersConfiguration
from utils.utils_tx import NetworkProviders
from tools.notebooks import env
from contracts.liquid_locking_contract import LiquidLockingContract
from contracts.builtin_contracts import SFControlContract

config = DefaultTransactionBuildersConfiguration(chain_id="1")
proxy = ProxyNetworkProvider("https://proxy-shadowfork-one.elrond.ro")
network_provider = NetworkProviders("https://express-api-shadowfork-one.elrond.ro", proxy.url)

In [ ]:
contract = LiquidLockingContract([], "")
sf_control = SFControlContract(env_config.SF_CONTROL_ADDRESS)

In [ ]:
from multiversx_sdk import AccountNonceHolder, UserSigner
from pathlib import Path

user = env.USER1
owner = env.OWNER
user_nonce_holder = AccountNonceHolder(proxy.get_account(user).nonce)
user_signer = UserSigner.from_pem_file(Path(env.USER1_PEM))

In [ ]:
from utils.utils_chain import Account
from multiversx_sdk import AccountNonceHolder, UserSigner

minter = Account(pem_file="~/Documents/sh1.pem", pem_index=0)
owner = Account(pem_file="~/Documents/sh1.pem")
user = Account(pem_file="~/Documents/sh1.pem")
user_nonce_holder = AccountNonceHolder(proxy.get_account(minter.address).nonce)
minter.sync_nonce(proxy)
owner.sync_nonce(proxy)
user.sync_nonce(proxy)

Issue metaesdt tokens

In [ ]:
import time
from contracts.builtin_contracts import ESDTContract
esdt_contract = ESDTContract(env_config.TOKENS_CONTRACT_ADDRESS)
nr_tokens = 2
token_ticker = "LIQUID"

# issue a number of nr_tokens tokens
hashes = []
for i in range(1, nr_tokens):
    hash = esdt_contract.issue_meta_esdt_token(owner, proxy, 
                                     [f"{token_ticker}{i}", f"{token_ticker}{i}", 18, 
                                      "canPause", "true", 
                                      "canFreeze", "true", "canWipe", "true", "canChangeOwner", "true", 
                                      "canUpgrade", "true", "canAddSpecialRoles", "true"])
    print(f"Issued {token_ticker}{i} in: {hash}")
    hashes.append(hash)
    time.sleep(1)

# check issue success and retrieve tokens
issued_collections = []
for hash in hashes:
    if network_provider.check_complex_tx_status(hash, "issue token"):
        tx_data = proxy.get_transaction(hash)
        issue_event = tx_data.logs.find_first_or_none_event("issue")
        issued_token = str(issue_event.topics[0])
        print(issue_event.topics[0].raw)
        print("Issued token:", issued_token)
        issued_collections.append(issued_token)
    else:
        print(f"Transaction {hash} failed!")

hashes = []
for collection in issued_collections:
    hashes.append(esdt_contract.set_special_role_token(owner, proxy, [collection, owner.address.bech32(), "ESDTRoleNFTCreate"]))
    time.sleep(1)
    hashes.append(esdt_contract.set_special_role_token(owner, proxy, [collection, owner.address.bech32(), "ESDTRoleNFTAddQuantity"]))
    time.sleep(1)
    hashes.append(esdt_contract.set_special_role_token(owner, proxy, [collection, owner.address.bech32(), "ESDTRoleNFTBurn"]))
    time.sleep(1)

# check special roles success
issued_collections = []
for hash in hashes:
    network_provider.check_complex_tx_status(hash, "set special roles")

for collection in issued_collections:
    

In [ ]:
tx_data = proxy.get_transaction(hash)
if tx_data.status.is_failed:
    print("Transaction failed")
    exit(1)
issue_event = tx_data.logs.find_first_or_none_event("issue")
issued_token = str(issue_event.topics[0])
print(issue_event.topics[0].raw)
print("Issued token:", issued_token)

Issue esdt tokens

In [ ]:
import time
from contracts.builtin_contracts import ESDTContract
esdt_contract = ESDTContract(env_config.TOKENS_CONTRACT_ADDRESS)
nr_tokens = 2
token_ticker = "LIQUID"

# issue a number of nr_tokens tokens
hashes = []
for i in range(1, nr_tokens+1):
    hash = esdt_contract.issue_fungible_token(owner, proxy, 
                                     [f"{token_ticker}{i}", f"{token_ticker}{i}", 1000000000000000000000000000, 18, 
                                      "canMint", "true", "canBurn", "true", "canPause", "true", "canFreeze", "true", "canWipe", "true", "canChangeOwner", "true", "canUpgrade", "true", "canAddSpecialRoles", "true"])
    print(f"Issued {token_ticker}{i} in: {hash}")
    hashes.append(hash)
    time.sleep(1)

# check issue success and retrieve tokens
issued_tokens = []
for hash in hashes:
    if network_provider.check_complex_tx_status(hash, "issue token"):
        tx_data = proxy.get_transaction(hash)
        issue_event = tx_data.logs.find_first_or_none_event("issue")
        issued_token = str(issue_event.topics[0])
        print(issue_event.topics[0].raw)
        print("Issued token:", issued_token)
        issued_tokens.append(issued_token)
    else:
        print(f"Transaction {hash} failed!")

print(f'Issued ESDT Tokens: {issued_tokens}')

In [ ]:
issued_tokens = ['LIQUID1-951a95', 'LIQUID2-951a93']

Send tokens

In [ ]:
import time

user_nonce_holder = AccountNonceHolder(proxy.get_account(minter.address).nonce)
hashes = []

for i in range(10):
    user_account = Account(pem_file="~/Documents/split/C10.pem", pem_index=i%10)
    minter.sync_nonce(proxy)
    
    transfer = []
    for token in issued_tokens:
        transfer.append(TokenPayment.fungible_from_amount(token, 1000, 18))

    builder = MultiESDTNFTTransferBuilder(
        config,
        destination=user_account.address,
        sender=minter.address,
        gas_limit=30000000,
        payments=transfer,
    )

    tx = builder.build()
    tx.nonce = user_nonce_holder.get_nonce_then_increment()
    tx.signature = minter.signer.sign(tx)
    hash = proxy.send_transaction(tx)
    print(hash)
    hashes.append(hash)
    time.sleep(2)

Send EGLD

In [ ]:
import time

user_nonce_holder = AccountNonceHolder(proxy.get_account(minter.address).nonce)
hashes = []

for i in range(10):
    user_account = Account(pem_file="~/Documents/split/C10.pem", pem_index=i%10)
    minter.sync_nonce(proxy)
    
    transfer = TokenPayment.egld_from_amount(1)

    builder = EGLDTransferBuilder(
        config,
        receiver=user_account.address,
        sender=minter.address,
        gas_limit=50000,
        payment=transfer,
    )

    tx = builder.build()
    tx.nonce = user_nonce_holder.get_nonce_then_increment()
    tx.signature = minter.signer.sign(tx)
    hash = proxy.send_transaction(tx)
    print(hash)
    hashes.append(hash)
    time.sleep(2)

Deploy contract

In [ ]:
from pathlib import Path
owner.sync_nonce(proxy)
bytecode = Path.home() / "projects" / "contracts" / "mx-contracts-rs" / "contracts" / "liquid-locking" / "output" / "liquid-locking.wasm"
hash, contract.address = contract.contract_deploy(owner, proxy, str(bytecode), [10])
print(f'Contract address: {contract.address}')

In [ ]:
import time
owner.sync_nonce(proxy)
for token in issued_tokens:
    contract.whitelist_token(owner, proxy, [token])
    time.sleep(2)

Lock tokens

In [ ]:
from utils.utils_tx import ESDTToken

user = Account(pem_file="~/Documents/split/C10.pem", pem_index=0)
user.sync_nonce(proxy)
token = [ESDTToken(issued_tokens[0], 0, 1000000000000000000),
         ESDTToken(issued_tokens[1], 0, 1000000000000000000)]
contract.lock(user, proxy, [token])

In [ ]:
locked_tokens = contract.get_locked_tokens(proxy, user.address.bech32())
print(locked_tokens)
locked_amounts = contract.get_locked_token_amounts(proxy, user.address.bech32())
print(locked_amounts)